In [1]:
#JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [46]:
bnf_1hr = pd.read_csv("../data/bnf_daily_tv.csv")
bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
bnf_1hr = bnf_1hr.drop(columns=["time"])
bnf_1hr = bnf_1hr[(bnf_1hr["datetime"].dt.year >= 2024)]
bnf_1hr.head()

,open,high,low,close,datetime
5775,48203.4490,48450.0000,48044.1480,48234.3010,2024-01-01
5776,48194.8010,48223.5510,47689.8010,47761.6480,2024-01-02
5777,47796.3010,47798.7500,47481.3520,47704.9490,2024-01-03
5778,47805.3980,48281.1990,47738.1480,48195.8520,2024-01-04
5779,48245.5510,48381.9490,47822.8520,48159.0000,2024-01-05


In [47]:
async def get_expiry(f_today):
    
    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24 :
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(start_date=nearest_wednesday, end_date=nearest_wednesday).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_expiry_nifty(f_today):
    
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [108]:
# spot_data = pd.read_csv('../data/bnf_min.csv')
# spot_data['datetime'] = pd.to_datetime(spot_data['datetime'])
# spot_data = pl.DataFrame(spot_data)
# spot_data = spot_data.with_columns([pl.col('datetime').alias('index')])

def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | str | None = None
) -> pl.DataFrame:
    
    if isinstance(offset, int):
        offset = dt.timedelta(days=offset)

    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            truncate=True,
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )

# bnf_resampled = resample(spot_data, '60m', pd.Timedelta(minutes=15))

# bnf_df = bnf_resampled.to_pandas()
# bnf_1hr = bnf_df

In [109]:
PORTFOLIO = 10_00_000
INDEX_LEVERAGE = 4
SLIPPAGE_FACTOR = 0.0003

In [110]:
bnf_1hr['datetime'] = pd.to_datetime(bnf_1hr['datetime'])
set_of_dates = set(bnf_1hr['datetime'].dt.date)
list_of_dates = list(set_of_dates)
print(list_of_dates[0])

2024-05-17


In [111]:
# start_date = dt.date(2017, 5, 1)
# end_date = dt.date(2024, 6, 9)

# bnf_df = await fetch_spot_data(
#     instrument='bnf',
#     start_date=start_date,
#     start_time=dt.time(9, 15),
#     end_date=end_date,
#     end_time=dt.time(15, 30)
# )
bnf_df = pd.read_csv('../data/bnf_min.csv')
print(bnf_df.tail())
bnf_df = pl.DataFrame(bnf_df)
bnf_df = bnf_df.with_columns(pl.col('datetime').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%.f', strict=False))
bnf_daily = resample(bnf_df, '1d')
bnf_pd = bnf_daily.to_pandas()
bnf_pd.tail()

                          datetime       open       high        low  \
670272  2024-03-28T15:25:00.000000 47146.3500 47165.8000 47128.7000   
670273  2024-03-28T15:26:00.000000 47143.9000 47151.9000 47128.5000   
670274  2024-03-28T15:27:00.000000 47134.7000 47150.2500 47124.1000   
670275  2024-03-28T15:28:00.000000 47150.2500 47162.3500 47124.0000   
670276  2024-03-28T15:29:00.000000 47137.1500 47155.2000 47121.2000   

            close  volume  
670272 47149.7500       0  
670273 47136.0000       0  
670274 47150.0000       0  
670275 47126.5500       0  
670276 47146.7500       0  


/tmp/ipykernel_4151/3021377870.py:22: DeprecationWarning: `truncate` is deprecated and will be removed in a future version. Please replace `truncate=True` with `label='left'` to silence this warning.
  .agg(


,datetime,open,high,low,close,volume
1790,2024-03-21,46674.8500,46990.2500,46570.1500,46679.2500,0
1791,2024-03-22,46634.9000,46974.1500,46566.8000,46849.0500,0
1792,2024-03-26,46552.9500,46788.3500,46529.0500,46617.4500,0
1793,2024-03-27,46643.4500,46956.1000,46643.4500,46851.1500,0
1794,2024-03-28,46827.8500,47440.4500,46827.8500,47146.7500,0


In [135]:
async def execute(df):
    tradebook = []
    
    current_date = start_date

    index = 'bnf'

    away_pct = 0

    df['datetime'] = pd.to_datetime(df['datetime'])

    # print(df)

    for i in range(0, len(df)-1):
        # print(df.iloc[i])
        dt_index = (df.iloc[i]['datetime']).date()
        next_dt_index = (df.iloc[i+1]['datetime']).date()
        
        print(dt_index)

        nearest_expiry = await get_expiry(next_dt_index)
        dte = (nearest_expiry - dt_index).days

        away_pct = dte * 0.2
        
        spot_df = await fetch_spot_data(
            instrument=index,
            start_date=dt_index,
            end_date=dt_index,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 20),
        )
        
        
        if not isinstance(spot_df, str):
            spot_df = spot_df.to_pandas()
            spot_price_at_entry = spot_df.iloc[0]['c']
            last_spot_close = spot_df.iloc[-1]['c']
            day_high = max(spot_df['h'])
            day_low = min(spot_df['l'])
        else:
            continue

        # call_strike = int(round(last_spot_close * (1 + (away_pct/100)) / 100) * 100)
        put_strike = int(round(last_spot_close * (1 - (away_pct/100)) / 100) * 100)
        

        # ce_df = await fetch_option_data(
        #     index=index,
        #     expiry=nearest_expiry,
        #     strike=call_strike,
        #     asset_class="C",
        #     start_date=dt_index,
        #     start_time=dt.time(15, 25),
        #     end_date=next_dt_index,
        #     end_time=dt.time(9, 15)
        # )
        
        # if not isinstance(ce_df, str):
        #     ce_df = ce_df.to_pandas()
        #     entry_price_ce = ce_df.iloc[0]['c']
        #     exit_price_ce = ce_df.iloc[-1]['c']
        # else:
        #     entry_price_ce = 0
        #     exit_price_ce = 0

        pe_df = await fetch_option_data(
            index=index,
            expiry=nearest_expiry,
            strike=put_strike,
            asset_class="P",
            start_date=dt_index,
            start_time=dt.time(15, 25),
            end_date=next_dt_index,
            end_time=dt.time(9, 15)
        )

        if not isinstance(pe_df, str):
            pe_df = pe_df.to_pandas()
            entry_price_pe = pe_df.iloc[0]['c']
            exit_price_pe = pe_df.iloc[-1]['c']
        else:
            entry_price_pe = 0
            exit_price_pe = 0

        qty = int(round(PORTFOLIO * INDEX_LEVERAGE / spot_price_at_entry) / 15) * 15
        # points = (entry_price_ce - exit_price_ce) + (entry_price_pe - exit_price_pe)
        points = entry_price_pe - exit_price_pe
        # slippage = 0.005 * (entry_price_ce + exit_price_ce + entry_price_pe + exit_price_pe)
        slippage = 0.005 * (entry_price_pe + exit_price_pe)
        final_points = points - slippage
        
        trade = {
            'Date Of Entry': dt_index,
            'Date Of Exit': next_dt_index,
            'Day High': day_high,
            'Day Low': day_low,
            'Index': 'BANKNIFTY',
            # 'CE Strike': call_strike,
            'PE Strike': put_strike,
            'Expiry': nearest_expiry,
            'DTE': dte,
            # 'CE Entry Price': entry_price_ce,
            # 'CE Exit Price': exit_price_ce,
            'PE Entry Price': entry_price_pe,
            'PE Exit Price': exit_price_pe,
            # 'CE Points': entry_price_ce - exit_price_ce,
            'PE Points': entry_price_pe - exit_price_pe,
            'Combined Points': points,
            'Slippage': slippage,
            'Final Points': final_points,
            'Qty': qty,
            'PnL w cs': final_points * qty,
            'ROI%': final_points * qty * 100 / PORTFOLIO,
        }

        tradebook.append(trade)

        # current_date += dt.timedelta(days=1)

    return pd.DataFrame(tradebook)

my_trades = await execute(bnf_pd)

2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-23
2017-01-24
2017-01-25
2017-01-27
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-03
2017-04-05
2017-04-06
2017-04-07
2017-04-10
2017-04-11
2017-04-12
2017-04-13
2017-04-17
2017-04-18
2017-04-19
2017-04-20
2017-04-21
2017-04-24
2017-04-25
2017-04-26
2017-04-27
2017-04-28
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-15
2017-05-16

In [136]:
my_trades

,Date Of Entry,Date Of Exit,Day High,Day Low,Index,PE Strike,PE Entry Price,PE Exit Price,PE Points,Combined Points,Slippage,Final Points,Qty,PnL w cs,ROI%
0,2017-01-02,2017-01-03,18248.2000,17845.3500,BANKNIFTY,18000,120.9000,144.1000,-23.2000,-23.2000,1.3250,-24.5250,210,-5150.2500,-0.5150
1,2017-01-03,2017-01-04,18115.0000,17831.7500,BANKNIFTY,18000,88.5000,78.1500,10.3500,10.3500,0.8333,9.5167,210,1998.5175,0.1999
2,2017-01-04,2017-01-05,18092.5000,17872.6000,BANKNIFTY,17900,65.0000,18.7500,46.2500,46.2500,0.4188,45.8312,210,9624.5625,0.9625
3,2017-01-05,2017-01-06,18163.9500,17979.7000,BANKNIFTY,18100,158.3000,121.1000,37.2000,37.2000,1.3970,35.8030,210,7518.6300,0.7519
4,2017-01-06,2017-01-09,18323.0000,18157.3500,BANKNIFTY,18300,140.0000,154.9500,-14.9500,-14.9500,1.4748,-16.4247,210,-3449.1975,-0.3449
5,2017-01-09,2017-01-10,18372.9000,18257.7500,BANKNIFTY,18300,117.8500,90.0000,27.8500,27.8500,1.0393,26.8107,210,5630.2575,0.5630
6,2017-01-10,2017-01-11,18436.9500,18277.2000,BANKNIFTY,18400,76.9500,33.9000,43.0500,43.0500,0.5543,42.4958,210,8924.1075,0.8924
7,2017-01-11,2017-01-12,18889.4500,18515.8000,BANKNIFTY,18800,60.2000,34.1000,26.1000,26.1000,0.4715,25.6285,210,5381.9850,0.5382
8,2017-01-12,2017-01-13,18965.7000,18806.2500,BANKNIFTY,18900,165.8500,141.0000,24.8500,24.8500,1.5343,23.3157,210,4896.3075,0.4896
9,2017-01-13,2017-01-16,18950.1500,18781.6000,BANKNIFTY,18900,122.1000,130.2000,-8.1000,-8.1000,1.2615,-9.3615,210,-1965.9150,-0.1966


In [137]:
stats_df5 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)

# Iterate over each year
new_tb = my_trades
new_tb['Date Of Entry'] = pd.to_datetime(new_tb['Date Of Entry'])
new_tb['Trade Year'] = new_tb['Date Of Entry'].dt.year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = new_tb[(new_tb["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df5.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df5["Total ROI"].sum()
overall_total_trades = stats_df5["Total Trades"].sum()
overall_win_rate = (new_tb["ROI%"] > 0).mean() * 100
overall_avg_profit = new_tb[new_tb["ROI%"] > 0]["ROI%"].mean()
overall_avg_loss = new_tb[new_tb["ROI%"] < 0]["ROI%"].mean()
overall_max_drawdown = (
    new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df5.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df5

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,28.6542,247,59.5142,0.4515,-0.3971,-6.2374,4.5940
2018,-20.3349,245,55.1020,0.5525,-0.8956,-26.8314,-0.7579
2019,-2.6121,244,51.2295,0.5668,-0.6174,-20.0089,-0.1305
2020,-36.1512,251,54.5817,1.7003,-2.4026,-83.6218,-0.4323
2021,-18.6487,247,57.4899,0.9350,-1.4421,-32.3435,-0.5766
2022,7.8484,247,61.5385,1.0466,-1.5919,-26.5768,0.2953
2023,-3.8300,245,51.8367,0.4786,-0.6461,-12.6988,-0.3016
2024,-2.6556,61,44.2623,0.5349,-0.5895,-6.7436,-0.3938
Overall,-47.7298,1787.0000,55.5120,0.8184,-1.1295,-112.0607,-0.4259


In [138]:
my_trades['PE Points'].sum()

-294.99999999999955

In [134]:
my_trades['CE Points'].sum()

KeyError: 'CE Points'